# Basics of simulation-based computational neuroscience


This notebook introduces the principles and techniques of simulation-based network neuroscience. Simulation-based insights are particularly helpful when analytical solutions to neural models are intractable - typically due to non-linearities in the governing differential equations. A classic example is the reset of the membrane potential following a spike in simple spiking neuron models, which introduces a discontinuity not amenable to closed-form analysis. 

In neuroscience we want to model neural behaviour, and we have a plethora of neuron models to chose from to do so. They each vary in complexity, biological realism and computational cost. In general it is true that the higher the complexity of the model, the more computational resources are needed to simulate the system. Therefore, biological realism comes at the cost of computational effeciency. 

At one end are **rate-based models**, which represent neural activity as continuous firing rates. These models are computationally efficient can also often be analysed analytically, making them suitable for large-scale network analyses. The rate-based workshop will give you more information and insight into this. 

At the other end are detailed conductance-based models like the **Hodgkin-Huxley model**, which require numerical integration due to their complexity. In between are simplified spiking models like the **Leaky-Integrate and Fire (LIF) neuron**, which preserve spike-based computation while remaining computationally manageable. To simulate sych systems, we need to use numerical integration techniques; the most widely-used is the Euler method, which is also employed by simulation software such as BRIAN.


# Numerical Integration: The Euler method

Models in computational neuroscience are predominantely based on **(Ordinary) Differential Equations** (ODEs). Differential equations define the change of a system's state with respect to a certain quantity. In neuroscience, we often want to understand how the state of a neuron changes over time given an external stimulus or neuromodulation. Therefore neural ODEs or neural models define the change of neural activity $x$ (e.g. the rate or the membrane potential) over time: 
\begin{align*}\frac{dx}{dt} = f(x(t))\end{align*}. 
where $x(t)$ denotes the neural state at time $t$ and $f(x(t))$ defines how the neural activity changes over the integration time step $t$. 

Given the differential equations of this form, we can simulate how the neural activity develops given an initial value $x(t=0)$ (also called the initial condition). We do this by calculating the successive neural states starting with our intial condition: 
\begin{align*} x(t=1) &= x(t=0) + f(x(t=0)) \\ x(t=2) &= x(t=1) + f(x(t=1)) \\ &... \\ x(t=T) &= x(t=T-1) + f(x(t=T-1))\end{align*}
with $T$ being the last timestep we want to simulate. Formally the Euler method can be thus defined as 
\begin{align*} x(t+Δt)=x(t)+Δt⋅f(x(t)) \end{align*}. 

where $Δt$ is the integration time step. A smaller $Δt$ yields more accurate approximations but increases the number of steps required and thus the computational load.

The graph below shows how the Euler method approximates a continuous trajectory. In this case we know the true form of $x(t)$ at each point. Therefore, we can see that if $Δt$ is large, the discrepancy between the true value and the approximation via the Euler method is large as well. Therefore, careful selection of $Δt$ is critical. For spiking neuron models, $Δt$= 0.1ms is a standard value. 

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

In [ ]:
def euler_method(dt,init, start, end, fun):

    steps = int((end-start)/dt)
    vals = np.arange(start, end+dt, dt)
    vals[0] = init
    for ind, v in enumerate(vals[:-1]):
        vals[ind+1] = vals[ind] + dt * fun(vals[ind])

    return vals

def ODE(x):
    return 2 * x

def exact_solution(t):
    return init_x * np.exp(2 * t)

start = 0
end = 1
init_x = 2

# Function that generates and plots data based on the slider value
def update_plot(value):

    t = np.arange(start, end+value, value) # Example function using the slider value
    x_approx = euler_method(value, init_x, start, end, ODE)

    plt.figure(figsize=(6, 4))
    plt.plot(t, exact_solution(t),color = 'grey')
    plt.plot(t, x_approx, color = 'black', label = 'approximation')
    plt.plot([0,value],[init_x,x_approx[0]], color = 'red', label = 'dt')
    plt.xlabel('t')
    plt.ylabel('x')
    plt.xlim(start, end)
    plt.ylim(1.5, 15)
    plt.title(f'Euler Integration with dt = {value:.3f}')
    plt.legend()
    plt.grid(True)
    plt.show()

# Create a slider
slider = widgets.FloatSlider(value=0.1, min=0.01, max=0.2, step=0.01, description="dt:")

# Use interactive to update the plot when the slider moves
interactive_plot = widgets.interactive_output(update_plot, {'value': slider})

# Display slider and interactive plot
display(slider, interactive_plot)

FloatSlider(value=0.1, description='dt:', max=0.2, min=0.01, step=0.01)

Output()

# Neuronal differential equations

In this workshop you'll encounter two different types of neuron models: **rate-based neurons** and **spiking neurons**. As mentioned earlier, rate models can often be treated analytically, while spiking models can not. To see the differences in their formulation, we will demonstrate how to simulate them both using the Euler method. 

## Rate-based neurons

A rate model abstracts the activity of a neuron as a continuous-valued _firing rate_, described by a single ODE. A typical rate neuron model is the following:

$\tau \frac{dr}{dt} = -r + I(t)$

where $r(t)$ is the firing rate of the neuron at time $t$, $\tau$ is the time constant of integration, and $I(t)$ is an external input current or stimulus which can be turned on or off.

We can rearrange and discretise this equation using the Euler method: 

\begin{align*} 
r(t + \Delta t) = r(t) + \Delta t \cdot \left( \frac{-r(t) + I(t)}{\tau} \right)
\end{align*}

This allows us to simulate the time evolution of the firing rate numerically. Below is an interactive example. Use the sliders to explore how the firing rate responds to different stimulus strengths and durations.

In [5]:
def rate_neuron(r, I, tau = 10):
    return (-r + I)/tau

def integrate(neuron, time, stimulus, dt = 0.1):
    sim_time = int(time/dt)
    r_act = np.zeros([sim_time+1]) #init activation is 0 here
    t_record = np.zeros([sim_time])
    I = 0
    I_record =np.zeros([sim_time])

    for t in range(sim_time):
        if t > int(10/dt) and t < int(stimulus[1]/dt):
            I = stimulus[0]
        else:
            I = 0
        r_act[t+1] = r_act[t]+ dt * (rate_neuron(r_act[t],I))
        t_record[t] = dt * t
        I_record[t] = I

    return r_act[:-1], t_record, I_record


def update_rateplot(I_strength, I_duration ):

    rates, ts, stim = integrate(rate_neuron, 100, [I_strength, I_duration]) #our time is in ms here

    plt.figure(figsize=(6, 4))
    plt.plot(ts, rates,label = 'r')
    plt.plot(ts, stim, color = 'grey', label = 'I')
    plt.xlabel('t [ms]')
    plt.ylabel('r [Hz]')
    plt.title('Rate Neuron')
    plt.legend()
    plt.grid(True)
    plt.show()

# Create a slider
slider = widgets.FloatSlider(value=10, min=0, max=100, step=1, description="strength:")
slider_l = widgets.FloatSlider(value=50, min=1, max=90, step=1, description="length[ms]:")

# Use interactive to update the plot when the slider moves
interactive_plot = widgets.interactive_output(update_rateplot, {'I_strength': slider,'I_duration': slider_l})

# Display slider and interactive plot
display(slider, slider_l, interactive_plot)

FloatSlider(value=10.0, description='strength:', step=1.0)

FloatSlider(value=50.0, description='length[ms]:', max=90.0, min=1.0, step=1.0)

Output()

A **spiking neuron model** differs from a rate-based model by explicitly modeling the generation of discrete spikes. In the **Leaky-Integrate and Fire (LIF) model**, this is implemented using a spike threshold and reset: when the neuron's membrane potential $v(t)$ surpasses a defined spike threshold $\Theta_s$, a spike is said to occur, and the membrane potential is set to a reset potential - typically the resting potential $E_l$. 

This reset introduces a non-linearity to the system: the value of the membrane-potential after the reset does not follow linearly or continuously from its prior trajectory. As a result, the dynamics must be simulated numerically.

Mathematically the LIF neuron is defined as:

\begin{align*} \tau \frac{dv}{dt} = (E_l - v) + I(t) \\
if \ \ v(t) > \Theta_s: v(t) \leftarrow E_l \end{align*}

where $v$ is the neuron's membrane potential, $E_l$ is the resting potential, $\tau$ is the membrane time constant, $I(t)$ is the external input current, and $\Theta_s$ is the spiking threshold.

This minimal model captures the essential feature of spike generation. More sophisticated variants exist which include, for example, differential equations for after-spike reset currents to model refractory periods, and adaptive thresholds to model spike-threshold adaption.

Below you can vary the received input to visualise its effect on the subthreshold membrane dynamics and the resulting spiking pattern. 


In [6]:
def LIF_neuron(v, I, Theta_s=-50, E_L=-70, tau=10):
    if v < Theta_s:
        return ((E_L - v) + I) / tau
    else:
        return E_L  # immediate reset, we do not approximate the spike form here

def integrate(neuron, time, stimulus, dt=0.1):
    sim_time = int(time / dt)
    v_act = np.zeros(sim_time + 1) - 70  # initial potential set to E_L
    t_record = np.zeros(sim_time)
    I_record = np.zeros(sim_time)

    for t in range(sim_time):
        if t > int(10 / dt) and t < int(stimulus[1] / dt):  # define stimulus window
            I = stimulus[0]
        else:
            I = 0
        v_act[t + 1] = v_act[t] + dt * neuron(v_act[t], I)
        t_record[t] = dt * t
        I_record[t] = I

    return v_act[:-1], t_record, I_record, np.nonzero(v_act[:-1] >= -50)[0]

def update_rateplot(I_strength, I_duration):
    vs, ts, stim, spikes = integrate(LIF_neuron, 100, [I_strength, I_duration])  # time in ms

    plt.figure(figsize=(6, 4))
    plt.plot(ts, vs, label='v')
    plt.scatter(ts[spikes], np.full_like(spikes, -45), color='r')
    plt.plot(ts, stim, color='grey', label='I')
    plt.xlabel('t [ms]')
    plt.ylabel(r'$V_m$ [mV]')
    plt.title('LIF Neuron')
    plt.legend()
    plt.grid(True)
    plt.show()

slider = widgets.FloatSlider(value=10, min=0, max=100, step=1, description="strength:")
slider_l = widgets.FloatSlider(value=50, min=1, max=90, step=1, description="length [ms]:")

interactive_plot = widgets.interactive_output(update_rateplot, {'I_strength': slider, 'I_duration': slider_l})

display(slider, slider_l, interactive_plot)


FloatSlider(value=10.0, description='strength:', step=1.0)

FloatSlider(value=50.0, description='length [ms]:', max=90.0, min=1.0, step=1.0)

Output()

# Noise

To drive neural activity, one can either inject a constant external stimulus or noise into the neuron. Simulating noise has the advantage that it approximates the aggregate synaptic input from many presynaptic neurons that one does not need to model explicitly. 

A commonly-used form of noisy current $\eta(t)$ is modelled as an **Ornstein-Uhlenbeck (OU) process**. OU noise resembles a random walk with a restoring force: it integrates a Wiener process $\frac{dW}{dt}$ (time-step-adjusted white noise) over time. It is defined as: 
\begin{align*}
\tau \frac{d\eta}{dt} = \theta \cdot (\mu - \eta) + \sigma \underbrace{\frac{dW}{dt}}_{\text{Wiener process}} 
\end{align*}
 where:
 - $\mu$ is the mean value around which the process fluctuates
 - $\theta$ controls the rate at which $\eta(t)$ is pulled back towards $\mu$
 - $\sigma$ sets the amplitude of the fluctuations (the standard deviation)
 - $\tau$ determines the characteristic timescale of the fluctuations
 
The larger the $\sigma$, the greater the deviations from the mean. The larger the $\tau$, the slower the fluctuations. The drive $\theta$ governs how fast the OU process returns to $\mu$ after a perturbation. 

In practice, we simulate this stochastic differential equation using the Euler-Maruyama method, which gives the following equation:
\begin{align*}
\eta(t+1) = \eta(t) + \frac{\Delta t}{\tau} \theta (\mu - \eta) + \sigma \underbrace{\sqrt{\frac{2 \Delta t}{\tau}}\cdot N(0,1)}_{\text{Wiener process}}
\end{align*}

The square-root scaling of the Wiener term is necessary to ensure that the magnitude of the fluctuations remains constant regardless of the spread $\sigma$ and the integration time step $\tau$.

Below, you can explore how the activity of a rate neuron and a LIF neuron differ depending on the **mean** and **standard deviation** of the OU noise. For simplicity, we set $ \theta = 1$, which is therefore omitted from the code.

In [ ]:
def integrate(time, stimulus, dt=0.1, tau=20):
    rng = np.random.RandomState(seed=5)  # fixed seed for reproducibility
    sim_time = int(time / dt)

    v_act = np.zeros(sim_time + 1) - 70  # initial membrane potential (E_L)
    r_act = np.zeros(sim_time + 1)       # initial rate activity
    t_record = np.zeros(sim_time)        # time array
    I = np.full(sim_time + 1, stimulus[0])  # initialize at mean value

    for t in range(sim_time):
        I[t + 1] = I[t] + (dt / tau) * (stimulus[0] - I[t]) + np.sqrt(2 * dt / tau) * stimulus[1] * rng.randn()
        v_act[t + 1] = v_act[t] + dt * LIF_neuron(v_act[t], I[t])
        r_act[t + 1] = r_act[t] + dt * rate_neuron(r_act[t], I[t])
        t_record[t] = dt * t

    return v_act[:-1], r_act[:-1], t_record, I[:-1]


def update_noiseplot(mean, sigma):
    vs, rs, ts, stim = integrate(100, [mean, sigma])  # simulation over 100 ms

    fig, ax = plt.subplots(3, 1, figsize=(6, 6))

    ax[0].plot(ts, stim)
    ax[0].plot(ts, np.full_like(ts, mean), color='grey', linestyle=':')
    ax[0].set_xlabel('t [ms]')
    ax[0].set_ylabel(r'$\eta(t)$  [mV]')
    ax[0].set_title('OU Noise')
    ax[0].grid(True)

    ax[1].plot(ts, rs)
    ax[1].plot(ts, np.zeros_like(ts), color='grey', linestyle=':')
    ax[1].set_xlabel('t [ms]')
    ax[1].set_ylabel(r'$r (t)$  [Hz]')
    ax[1].set_title('Rate Neuron')
    ax[1].grid(True)

    ax[2].plot(ts, vs)
    ax[2].plot(ts, np.full_like(ts, -70), color='grey', linestyle=':')
    ax[2].set_xlabel('t [ms]')
    ax[2].set_ylabel(r'$V_m$  [mV]')
    ax[2].set_title('LIF Neuron')
    ax[2].grid(True)

    fig.tight_layout()

# Create a slider
slider_mu = widgets.FloatSlider(value=0, min=-20, max=100, step=1, description="μ:")
slider_sigma = widgets.FloatSlider(value=2, min=0.1, max=20, step=0.1, description="σ:")

# Use interactive to update the plot when the slider moves
interactive_plot = widgets.interactive_output(update_noiseplot, {'mean': slider_mu, 'sigma': slider_sigma})

# Display slider and interactive plot
display(slider_mu, slider_sigma, interactive_plot)

FloatSlider(value=0.0, description='μ:', min=-20.0, step=1.0)

FloatSlider(value=2.0, description='σ:', max=20.0, min=0.1)

Output()

# Simulation modules & packages

So far, we have programmed every detail of the system ourselves. However, this is not strictly neccessary. There are plenty of Python packages specifically designed for simulating dynamical systems and neural networks. This means that you don't have to take care of implementational details, but rather just to define your model, network architecture, and - if wished - integration method. This allows you to simulate large networks without having to manually optimise for computational performance, since the packages are already tuned for speed and scaleability. 

Using dedicated simulation libraries can mitigate the trade-off between model complexity and computational cost. Choosing the right package can help speed up your simulations significantly; some options include **Neuron**, **Nest**, **BRIAN**, and **Dendrify**, to name a few. 

In the following section, we'll show you how to implement the above noisy neuron using **BRIAN**. During the **Spiking Neural Network Tutorial** we'll continue to use this package, since its intuitive syntax makes network configuration easy to follow.

In [8]:
from brian2 import *
from brian2tools import *

To define a model in **BRIAN**, you have to specify it as a differential equation of the form: 

```differential_equation = dx/dt = dif(x) : unit```

The *unit* must always be given in SI units. **BRIAN** automatically checks your differential equations for unit compatibility. This means you'll get an error if, for example, an equation defining membrane potential dynamics does not conform to the expected units of volts per second $(V/s)$, or if you try to add a variable in Hertz to one in Amperes. This feature is particularly helpful when converting between units.

However, you don't have to specify units; you can also define **dimensionless** variables by assigning them a unit of $1$. 

You can add neural attributes by specifying the variable after the differential equation:
```variable:unit```

For example, you can specify the membrane time constant this way, and give each neuron in your network a different time constant by setting it explicitly. Below, you'll find the definitions for:
- a dimensionless rate neuron,
- a rate neuron whose activity is measured explicitly in Hertz,
- a spiking LIF neuron.

Any variable that depends on time - like ```I_rate(t), I[t], I_spike(t)``` - is treated as an external, time-dependent input. These inputs are not part of the neuron's internal dynamics, but are provided separately. They can be used to deliver square pulses or repeated stimulus patterns into the neuron, without having to stop the simulation.

In [ ]:
# Differential equations
# Dimensionless rate neuron
diff_eq_rate = '''dr/dt = (-r + I_rate(t))/tau : 1
                  tau : second'''

# Frequency rate neuron 
diff_eq_rate_Hz = '''dr/dt = (-r + I[t])/tau : Hz
                     tau : second'''

# LIF neuron
diff_eq_spike = '''dv/dt = ((E_L - v) + I_spike(t))/tau : volt (unless refractory)
                   E_L : volt
                   v_spike : volt
                   tau : second'''


def run_brian_neurons(length, strength):
    start_scope()  # clears previous Brian objects to avoid duplication

    # Define neurons
    N_rate = NeuronGroup(1, diff_eq_rate, method='euler')
    N_rate.tau = 10 * ms

    N_spike = NeuronGroup(1, diff_eq_spike,
                          threshold='v > v_spike',
                          reset='v = E_L',
                          refractory=3 * ms,
                          method='euler')
    N_spike.E_L = -79 * mV
    N_spike.v_spike = -49 * mV
    N_spike.v = -79 * mV  # set initial condition (we need to define v(0) otherwise it is assumed to be 0)
    N_spike.tau = 10 * ms

    # Define input stimulus (200 ms total duration, stimulus begins at 10ms)
    input_arr = np.zeros(200)
    onset = 10  # ms
    input_arr[onset:int(length) + onset] = strength

    # Define time-dependent input as TimedArray
    I_rate = TimedArray(input_arr, dt=1 * ms)   # each entry in the TimedArray lasts for 1ms
    I_spike = TimedArray(input_arr * mV, dt=1 * ms)   # needs to be in mV to be compatible with dimensions of the model (with proper consideration of conductances we could also inject a current here)

    # To record neuron activity, we need to define monitors
    # Syntax: StateMonitor(group to record from, variable to record, which neurons to record from, recording frequency)
    s_v = SpikeMonitor(N_spike)
    record_r = StateMonitor(N_rate, 'r', record=True, dt=1 * ms)
    record_v = StateMonitor(N_spike, 'v', record=True, dt=1 * ms)
    record_I = input_arr 

    run(500 * ms, report='text') # Run the network for 500ms; you can delete the status report by omitting the second argument in the function

    return s_v, record_r, record_v, record_I


def update_brianplot(stim_l, stim_s):
    spikemon, rec_r, rec_v, input_arr = run_brian_neurons(stim_l, stim_s)

    t_input = np.arange(len(input_arr))  # 0 to 199 ms

    fig, ax = plt.subplots(3, 1, figsize=(6, 7))

    # Plot stimulus
    ax[0].plot(t_input, input_arr, color='grey')
    ax[0].set_ylabel('I')
    ax[0].set_title('Input Current')
    ax[0].grid(True)

    # Plot rate neuron
    ax[1].plot(rec_r.t / ms, rec_r.r[0])
    ax[1].set_xlabel('t [ms]')
    ax[1].set_ylabel(r'$r$')
    ax[1].set_title('Rate Neuron')
    ax[1].grid(True)

    # Plot LIF neuron
    ax[2].plot(rec_v.t / ms, rec_v.v[0] / mV)
    ax[2].scatter(spikemon.t / ms, np.full_like(spikemon.t, -45), color='r') # If you want to plot a spike raster you can use SpikeMonitor
    ax[2].set_xlabel('t [ms]')
    ax[2].set_ylabel(r'$v$ [mV]')
    ax[2].set_title('LIF Neuron')
    ax[2].grid(True)

    fig.tight_layout()


# Create sliders
slider = widgets.FloatSlider(value=10, min=0, max=100, step=1, description="duration:")
slider_l = widgets.FloatSlider(value=50, min=0, max=100, step=0.1, description="amplitude:")

# Set up interactivity
interactive_plot = widgets.interactive_output(update_brianplot, {'stim_l': slider, 'stim_s': slider_l})

# Display slider and interactive plot
display(slider, slider_l, interactive_plot)


FloatSlider(value=10.0, description='duration:', step=1.0)

FloatSlider(value=50.0, description='amplitude:')

Output()